In [2]:
import numpy as np
import cv2

In [3]:

# from skimage import data, filters

# Open Video
cap = cv2.VideoCapture('cctv.mp4')

# Randomly select 25 frames
frameIds = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=25)

# Store selected frames in an array
frames = []
for fid in frameIds:
    cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
    ret, frame = cap.read()
    frames.append(frame)

# Calculate the median along the time axis
medianFrame = np.median(frames, axis=0).astype(dtype=np.uint8)    

# Display median frame
cv2.imshow('frame', medianFrame)
cv2.waitKey(0)

# Reset frame number to 0
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Convert background to grayscale
grayMedianFrame = cv2.cvtColor(medianFrame, cv2.COLOR_BGR2GRAY)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)

result = cv2.VideoWriter('filename.mp4',
						cv2.VideoWriter_fourcc(*'MJPG'),
						30, size)

# Loop over all frames
ret = True
while(ret):

    # Read frame
    ret, frame = cap.read()
    # Convert current frame to grayscale
    if ret:
        frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Calculate absolute difference of current frame and 
        # the median frame
        dframe = cv2.absdiff(frame_g, grayMedianFrame)
        # Treshold to binarize
        th, dframe = cv2.threshold(dframe, 60, 255, cv2.THRESH_BINARY) # adjust threshold according to the video
        # Save image
        count = 0
        choose = False
        n_white_pix = np.sum(dframe == 255)
        if n_white_pix > 0.15*frame_height*frame_width:
            choose = True
        if choose:   
            to_BGR = cv2.cvtColor(dframe, cv2.COLOR_GRAY2BGR)
            result.write(frame)


# Release video object
cap.release()
result.release()
# Destroy all windows
cv2.destroyAllWindows()
